In [1]:
import transformers 
import datasets
import torch
import logging

c:\Users\dama_\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Preprocessing
# sep_token = '<sep>'
dataset_name = "squad"
model_type="electra"
model_name= "google/electra-base-discriminator"
models_dir = "saved_models/electra-base-discriminator_squad_mod"
checkpoint = 'electra'
max_input_length = 386


# ## Training
learning_rate = 3e-5
num_epochs = 3

In [3]:
dataset=torch.load("../datasets/squad.pt")

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

In [4]:
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)

In [5]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_input_length ,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [6]:
tokenized_squad = dataset.map(preprocess_function, batched=True, remove_columns=dataset["train"].column_names)

Loading cached processed dataset at C:\Users\dama_\.cache\huggingface\datasets\squad\plain_text\1.0.0\d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453\cache-87877fe25e986fb6.arrow
Loading cached processed dataset at C:\Users\dama_\.cache\huggingface\datasets\squad\plain_text\1.0.0\d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453\cache-51f0916044623eae.arrow


In [7]:
data_collator = transformers.DefaultDataCollator()

In [8]:
model = transformers.AutoModelForQuestionAnswering.from_pretrained(model_name)

Some weights of the model checkpoint at google/electra-base-discriminator were not used when initializing ElectraForQuestionAnswering: ['discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias']
- This IS expected if you are initializing ElectraForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForQuestionAnswering were not initialized from the model checkpoint at google/electra-base-discriminator and are newly initialized: ['qa_outputs.weight', 'qa_outputs.

In [9]:
training_args = transformers.TrainingArguments(
    output_dir=models_dir,
    evaluation_strategy="epoch",
    learning_rate=learning_rate,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    push_to_hub=True,
)

trainer = transformers.Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_squad["train"],
    eval_dataset=tokenized_squad["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

e:\HOGENT\2022_2023\BA\BP_Info_Support\bert\saved_models/electra-base-discriminator_squad_mod is already a clone of https://huggingface.co/damapika/electra-base-discriminator_squad_mod. Make sure you pull the latest changes with `repo.git_pull()`.


In [10]:
torch.cuda.memory_summary(device=None, abbreviated=False)
torch.cuda.empty_cache()

In [11]:
trainer.train()

c:\Users\dama_\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: damapika. Use `wandb login --relogin` to force relogin


  2%|▏         | 500/32850 [17:54<19:34:07,  2.18s/it]

{'loss': 2.1018, 'learning_rate': 2.954337899543379e-05, 'epoch': 0.05}


  3%|▎         | 1000/32850 [36:03<19:19:25,  2.18s/it]

{'loss': 1.2541, 'learning_rate': 2.908675799086758e-05, 'epoch': 0.09}


...

Several commits (36) will be pushed upstream.
 99%|█████████▉| 32500/32850 [14:12:40<09:57,  1.71s/it]  

{'loss': 0.4249, 'learning_rate': 3.19634703196347e-07, 'epoch': 2.97}


Several commits (37) will be pushed upstream.
                                                        
100%|██████████| 32850/32850 [14:33:57<00:00,  1.60s/it]

{'eval_loss': 1.0307191610336304, 'eval_runtime': 655.4424, 'eval_samples_per_second': 16.127, 'eval_steps_per_second': 2.017, 'epoch': 3.0}
{'train_runtime': 52440.3864, 'train_samples_per_second': 5.011, 'train_steps_per_second': 0.626, 'train_loss': 0.6815786130025506, 'epoch': 3.0}


TrainOutput(global_step=32850, training_loss=0.6815786130025506, metrics={'train_runtime': 52440.3864, 'train_samples_per_second': 5.011, 'train_steps_per_second': 0.626, 'train_loss': 0.6815786130025506, 'epoch': 3.0})

In [18]:
trainer.push_to_hub()

Several commits (39) will be pushed upstream.
The progress bars may be unreliable.
Upload file pytorch_model.bin: 427MB [22:08, 11.5MB/s]                            To https://huggingface.co/damapika/electra-base-discriminator_squad_mod
   53f9db0..a45c6dc  main -> main

Upload file pytorch_model.bin: 100%|██████████| 415M/415M [22:10<00:00, 327kB/s] 
Upload file runs/May01_14-59-48_Damapika/events.out.tfevents.1682945996.Damapika.26664.0: 100%|██████████| 15.3k/15.3k [02:19<00:00, 113B/s]  


In [19]:
question = "Who is Fyodor Dostoevsky?"
context = "In the world of literature, there have been many authors who have gained a reputation for their ability to create complex characters. One such author is Fyodor Dostoevsky, a Russian novelist who wrote several influential works in the 19th century."

In [20]:
question_answerer = transformers.pipeline("question-answering", model="damapika/electra-base-discriminator_squad_mod")
question_answerer(question=question, context=context)

{'score': 0.9061723351478577,
 'start': 172,
 'end': 190,
 'answer': 'a Russian novelist'}